In [1]:
%pip -q install numpy pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd

def clean_yn_columns(df: pd.DataFrame, columns: list):
    """
    지정한 컬럼들에 대해
    - 결측치 -> 'N'
    - Y/N -> 1/0
    """
    for col in columns:
        if col not in df.columns:
            raise ValueError(f"컬럼 없음: {col}")

        df[col] = (
            df[col]
            .fillna("N")            # NaN -> N
            .astype(str)
            .str.strip()            # 공백 제거
            .replace({"": "N"})     # 빈 문자열 -> N
            .map({"Y": 1, "N": 0})  # Y/N -> 1/0
        )

    return df


In [32]:
# load
crosswalks = pd.read_csv("./raw/crosswalks.csv", encoding="cp949")

crosswalk_cols = [
    "highland",
    "signal",
    "button",
    "sound_signal",
    "bump",
    "braille_block",
    "spotlight"
]

crosswalks = clean_yn_columns(crosswalks, crosswalk_cols)

# save
crosswalks.to_csv("crosswalks_passenger.csv", index=False,encoding="utf-8-sig")


In [33]:
# load
signals = pd.read_csv("./raw/signals.csv", encoding="cp949")

signal_cols = [
    "main_road",
    "button",
    "remain_time",
    "sound_signal"
]

signals = clean_yn_columns(signals, signal_cols)

# save
signals.to_csv("signals_passenger.csv", index=False,encoding="utf-8-sig")


In [34]:
import re
import numpy as np
import pandas as pd
from sklearn.neighbors import BallTree

EARTH_RADIUS_M = 6371000  # meters

# -----------------------------
# 1) CSV 로더 (cp949/utf-8-sig 자동)
# -----------------------------
def read_csv_any(path: str) -> tuple[pd.DataFrame, str]:
    for enc in ("utf-8-sig", "cp949", "euc-kr"):
        try:
            return pd.read_csv(path, encoding=enc), enc
        except Exception:
            pass
    # 마지막 수단
    return pd.read_csv(path, encoding="latin1"), "latin1"

# -----------------------------
# 2) 크로스워크-신호등 매핑 (BallTree + haversine)
# -----------------------------
def build_crosswalk_signal_map(
    signal_df: pd.DataFrame,
    crosswalk_df: pd.DataFrame,
    R_m: float = 30,
    top_k: int = 2,
    use_admin_filter: bool = True,
) -> pd.DataFrame:
    sg = signal_df.copy()
    cw = crosswalk_df.copy()

    # required columns
    req_sg = {"signal_id", "signal_lat", "signal_lon"}
    req_cw = {"crosswalk_id", "crosswalk_lat", "crosswalk_lon"}
    missing = (req_sg - set(sg.columns)) | (req_cw - set(cw.columns))
    if missing:
        raise ValueError(f"Missing required columns: {sorted(missing)}")

    # numeric conversion
    for col in ("signal_lat", "signal_lon"):
        sg[col] = pd.to_numeric(sg[col], errors="coerce")
    for col in ("crosswalk_lat", "crosswalk_lon"):
        cw[col] = pd.to_numeric(cw[col], errors="coerce")

    sg = sg.dropna(subset=["signal_id", "signal_lat", "signal_lon"]).reset_index(drop=True)
    cw = cw.dropna(subset=["crosswalk_id", "crosswalk_lat", "crosswalk_lon"]).reset_index(drop=True)

    if len(sg) == 0 or len(cw) == 0:
        return pd.DataFrame(columns=["crosswalk_id", "signal_id", "distance_m", "confidence", "match_method", "sido", "sigungu"])

    sg_rad = np.deg2rad(sg[["signal_lat", "signal_lon"]].to_numpy())
    cw_rad = np.deg2rad(cw[["crosswalk_lat", "crosswalk_lon"]].to_numpy())

    tree = BallTree(sg_rad, metric="haversine")
    k = min(top_k, len(sg))
    dist_rad, idx = tree.query(cw_rad, k=k)
    dist_m = dist_rad * EARTH_RADIUS_M

    rows = []
    for i in range(len(cw)):
        for j in range(k):
            d = float(dist_m[i, j])
            if d > R_m:
                continue
            srow = sg.iloc[int(idx[i, j])]
            crow = cw.iloc[i]

            if use_admin_filter and all(x in sg.columns for x in ("sido", "sigungu")) and all(x in cw.columns for x in ("sido", "sigungu")):
                if str(srow["sido"]) != str(crow["sido"]) or str(srow["sigungu"]) != str(crow["sigungu"]):
                    continue

            conf = max(0.0, 1.0 - (d / float(R_m)))
            rows.append(
                {
                    "crosswalk_id": crow["crosswalk_id"],
                    "signal_id": srow["signal_id"],
                    "distance_m": round(d, 3),
                    "confidence": round(conf, 6),
                    "sido": crow.get("sido", srow.get("sido", None)),
                    "sigungu": crow.get("sigungu", srow.get("sigungu", None)),
                }
            )

    return pd.DataFrame(rows)


In [35]:
# -----------------------------
# 3) 실행: 필터링 + 저장 + 매핑 생성
# -----------------------------

signals_df, sig_enc = read_csv_any("signals_passenger.csv")
crosswalks_df, cw_enc = read_csv_any("crosswalks_passenger.csv")
print("loaded encodings:", {"signals": sig_enc, "crosswalks": cw_enc})
print("signals shape:", signals_df.shape, "crosswalks shape:", crosswalks_df.shape)

# (1) signals: signal_type == '02' (보행등)
signals_passenger = signals_df[signals_df["signal_type"].astype(str).str.zfill(2) == "02"].copy()

# (3) 매핑 생성은 좌표가 필요하므로 "drop 이전" 데이터로 수행
mapping_df = build_crosswalk_signal_map(
    signals_passenger,
    crosswalks_df,
    R_m=30,
    top_k=2,
    use_admin_filter=True,
)

# (5) 저장
signals_passenger.to_csv("signals_passenger.csv", index=False, encoding="utf-8-sig")
crosswalks_df.to_csv("crosswalks_passenger.csv", index=False, encoding="utf-8-sig")
mapping_df.to_csv("crosswalk_signal_map.csv", index=False, encoding="utf-8-sig")

loaded encodings: {'signals': 'utf-8-sig', 'crosswalks': 'utf-8-sig'}
signals shape: (50000, 15) crosswalks shape: (50000, 18)


In [ ]:
import pandas as pd
import numpy as np

district_df = pd.read_csv('district.csv', encoding='utf-8')
accident_df = pd.read_csv('passenger_accident.csv', encoding='cp949')

acc_in_district = accident_df['district_code'].dropna().unique()

dist_temp = district_df[district_df['district_code'].isin(acc_in_district)]

dist_temp['available'] = (
            dist_temp['available']
            .map({"존재": 1, "폐지": 0}) 
        )

dist_temp.to_csv('district.csv', index=False, encoding='utf-8-sig')

In [16]:
accident_df = pd.read_csv('passenger_accident.csv', encoding='cp949')

accident_df['year'] = (
    accident_df['year']
    .astype('string')
    .str.slice(0, 4)
    .astype('Int64') 
)

accident_df['detail'] = accident_df['detail'].astype('string').str.split('(').str[1].str.replace(")","")

accident_df.to_csv('passenger_accident.csv',index=False, encoding='utf-8-sig')



In [37]:
import pandas as pd

cw = pd.read_csv("crosswalks_passenger.csv", encoding="utf-8")
sg = pd.read_csv("signals_passenger.csv", encoding="utf-8")


In [38]:
CROSSWALK_ID_COL = "crosswalk_id"   # 예: "id" 인 경우 바꿔줘
SIGNAL_ID_COL    = "signal_id"      # 예: "id"


In [39]:
def check_id_uniqueness(df, id_col, name):
    total = len(df)
    unique = df[id_col].nunique(dropna=True)
    null_cnt = df[id_col].isna().sum()

    print(f"📌 {name}")
    print(f"  - total rows       : {total}")
    print(f"  - unique {id_col}  : {unique}")
    print(f"  - null {id_col}    : {null_cnt}")
    print(f"  - is unique?       : {total == unique and null_cnt == 0}")
    print("-" * 40)

check_id_uniqueness(cw, CROSSWALK_ID_COL, "crosswalks_passenger")
check_id_uniqueness(sg, SIGNAL_ID_COL, "signals_passenger")


📌 crosswalks_passenger
  - total rows       : 50000
  - unique crosswalk_id  : 37356
  - null crosswalk_id    : 0
  - is unique?       : False
----------------------------------------
📌 signals_passenger
  - total rows       : 16005
  - unique signal_id  : 13272
  - null signal_id    : 0
  - is unique?       : False
----------------------------------------


In [40]:
def show_duplicate_ids(df, id_col, top_n=20):
    dup = df[df[id_col].duplicated(keep=False)].sort_values(id_col)
    print(f"중복 {id_col} 개수:", dup[id_col].nunique())
    return dup.head(top_n)

dup_cw = show_duplicate_ids(cw, CROSSWALK_ID_COL)
dup_sg = show_duplicate_ids(sg, SIGNAL_ID_COL)

dup_cw, dup_sg


중복 crosswalk_id 개수: 3189
중복 signal_id 개수: 1235


(          sido sigungu                     address crosswalk_id  \
 15133      경기도     파주시       경기도 파주시 문산읍 당동리 888-4            1   
 37818     경상북도     상주시         경상북도 상주시 남성동 235-17            1   
 16509  전북특별자치도     진안군  전북특별자치도 진안군 주천면 대불리 2094-8            1   
 36836      경기도     이천시       경기도 이천시 신둔면 수광리 439-9            1   
 44577     경상남도     거제시        경상남도 거제시 사등면 사등리 480            1   
 15737     전라남도     무안군        전라남도 무안군 무안읍 성내리 357            1   
 26747    인천광역시     남동구         인천광역시 남동구 만수동 1-160            1   
 15623     경상북도   포항시북구     경상북도 포항시 북구 두호동 1109-15            1   
 3767      경상남도     김해시           경상남도 김해시 부원동 1041            1   
 17170    대전광역시     대덕구           대전광역시 대덕구 신탄진동 28            1   
 7660       경기도    남양주시          경기도 남양주시 금곡동 146-3            1   
 15142      경기도     파주시       경기도 파주시 문산읍 당동리 941-1           10   
 17128    대전광역시      중구           대전광역시 중구 탄방동 1461           10   
 7669       경기도    남양주시         경기도 남양주시 금곡동 185

In [41]:
def check_duplicate_coords(df, lat_col, lon_col, name):
    dup = df[df.duplicated(subset=[lat_col, lon_col], keep=False)]
    print(f"📍 {name} 동일 좌표 중복 행 수:", len(dup))
    return dup.head(10)

check_duplicate_coords(cw, "crosswalk_lat", "crosswalk_lon", "crosswalks_passenger")
check_duplicate_coords(sg, "signal_lat", "signal_lon", "signals_passenger")


📍 crosswalks_passenger 동일 좌표 중복 행 수: 13080
📍 signals_passenger 동일 좌표 중복 행 수: 4939


,sido,sigungu,road_type,road_direction,address,signal_lat,signal_lon,road_shape,main_road,signal_id,signal_type,button,remain_time,sound_signal,org_code
0,경상남도,거제시,지방도,3,경상남도 거제시 고현동 979-2,34.890263,128.624340,2,1,26,2,0,1,0,5370000
1,경상남도,거제시,지방도,3,경상남도 거제시 고현동 979-2,34.890263,128.624340,2,1,27,2,0,1,0,5370000
2,경상남도,거제시,지방도,3,경상남도 거제시 고현동 29-21,34.888842,128.625338,1,1,29,2,0,1,0,5370000
3,경상남도,거제시,지방도,3,경상남도 거제시 고현동 29-21,34.888842,128.625338,1,1,31,2,0,1,0,5370000
4,경상남도,거제시,지방도,3,경상남도 거제시 거제면 서정리 734-19,34.850209,128.585648,2,1,2,2,0,0,0,5370000
5,경상남도,거제시,지방도,3,경상남도 거제시 거제면 서정리 734-19,34.850209,128.585648,2,1,5,2,0,1,0,5370000
6,경상남도,거제시,지방도,3,경상남도 거제시 거제면 서정리 731-37,34.849511,128.586407,2,1,7,2,0,1,0,5370000
7,경상남도,거제시,지방도,3,경상남도 거제시 거제면 서정리 731-37,34.849511,128.586407,2,1,8,2,0,1,0,5370000
8,경상남도,거제시,지방도,3,경상남도 거제시 거제면 서정리 731-37,34.849511,128.586407,2,1,10,2,0,1,0,5370000
9,경상남도,거제시,지방도,3,경상남도 거제시 거제면 외간리 620-8,34.859008,128.575779,2,1,12,2,0,1,0,5370000


In [42]:
import pandas as pd

cw = pd.read_csv("crosswalks_passenger.csv", encoding="utf-8")
sg = pd.read_csv("signals_passenger.csv", encoding="utf-8")

# id 컬럼명
CW_ID_COL = "crosswalk_id"
SG_ID_COL = "signal_id"

# 1️⃣ 중복 제거 기준 정의 (좌표까지 동일하면 같은 객체)
cw_unique = cw.drop_duplicates(subset=[CW_ID_COL, "crosswalk_lat", "crosswalk_lon"]).copy()
sg_unique = sg.drop_duplicates(subset=[SG_ID_COL, "signal_lat", "signal_lon"]).copy()

# 2️⃣ surrogate id 생성
cw_unique["cw_uid"] = [
    f"CW_{i:06d}" for i in range(1, len(cw_unique) + 1)
]
sg_unique["sg_uid"] = [
    f"SG_{i:06d}" for i in range(1, len(sg_unique) + 1)
]

# 3️⃣ 저장
cw_unique.to_csv("crosswalks_passenger_unique.csv", index=False, encoding="utf-8-sig")
sg_unique.to_csv("signals_passenger_unique.csv", index=False, encoding="utf-8-sig")


In [43]:
map_df = pd.read_csv("crosswalk_signal_map.csv", encoding="utf-8")

map_df = (
    map_df
    .merge(cw_unique[[CW_ID_COL, "cw_uid"]], on=CW_ID_COL, how="left")
    .merge(sg_unique[[SG_ID_COL, "sg_uid"]], on=SG_ID_COL, how="left")
)

map_df.to_csv("crosswalk_signal_map_unique.csv", index=False, encoding="utf-8-sig")


In [44]:
map_df = pd.read_csv("crosswalk_signal_map_unique.csv", encoding="utf-8")
sg_df = pd.read_csv("signals_passenger_unique.csv", encoding='utf-8')
cw_df = pd.read_csv("crosswalks_passenger_unique.csv", encoding='utf-8')

map_df = map_df.drop(columns=['crosswalk_id', 'signal_id'], errors='ignore')
sg_df  = sg_df.drop(columns=['signal_id'], errors='ignore')
cw_df  = cw_df.drop(columns=['crosswalk_id'], errors='ignore')

map_df.to_csv("crosswalk_signal_map_unique.csv", index=False, encoding="utf-8-sig")
sg_df.to_csv("signals_passenger_unique.csv", index=False, encoding="utf-8-sig")
cw_df.to_csv("crosswalks_passenger_unique.csv", index=False, encoding="utf-8-sig")

In [5]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

# -----------------------
# 설정
# -----------------------
CROSSWALK_CSV = "crosswalks_passenger_unique.csv"
ACCIDENT_CSV  = "passenger_accident.csv"
OUT_CSV       = "acc_cw_within_100m.csv"

RADIUS_M = 100
EARTH_RADIUS_M = 6371000.0

def read_csv_auto(path, encodings=("utf-8", "cp949", "euc-kr")):
    last = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception as e:
            last = e
    raise last

def pick_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

cw = read_csv_auto(CROSSWALK_CSV)
acc = read_csv_auto(ACCIDENT_CSV)

# 컬럼 자동 탐색
cw_id  = pick_col(cw,  ["crosswalk_id", "id","cw_uid"])
cw_lat = pick_col(cw,  ["crosswalk_lat", "lat", "latitude", "위도"])
cw_lon = pick_col(cw,  ["crosswalk_lon", "lon", "longitude", "경도"])

acc_id  = pick_col(acc, ["accident_id", "id"])
acc_lat = pick_col(acc, ["accident_lat", "lat", "latitude", "위도"])
acc_lon = pick_col(acc, ["accident_lon", "lon", "longitude", "경도"])

need = [cw_id, cw_lat, cw_lon, acc_id, acc_lat, acc_lon]
if any(v is None for v in need):
    raise ValueError(
        "필수 컬럼을 찾지 못했어.\n"
        f"cw_id={cw_id}, cw_lat={cw_lat}, cw_lon={cw_lon}, "
        f"acc_id={acc_id}, acc_lat={acc_lat}, acc_lon={acc_lon}\n"
        f"crosswalk cols={list(cw.columns)}\n"
        f"accident cols={list(acc.columns)}"
    )

# 숫자 변환/결측 제거
for c in [cw_lat, cw_lon]:
    cw[c] = pd.to_numeric(cw[c], errors="coerce")
for c in [acc_lat, acc_lon]:
    acc[c] = pd.to_numeric(acc[c], errors="coerce")

cw_v = cw.dropna(subset=[cw_id, cw_lat, cw_lon]).reset_index(drop=True)
acc_v = acc.dropna(subset=[acc_id, acc_lat, acc_lon]).reset_index(drop=True)

# BallTree 준비 (라디안)
cw_rad  = np.deg2rad(cw_v[[cw_lat, cw_lon]].to_numpy())
acc_rad = np.deg2rad(acc_v[[acc_lat, acc_lon]].to_numpy())

tree = BallTree(cw_rad, metric="haversine")
radius_rad = RADIUS_M / EARTH_RADIUS_M

# 사고별 100m 내 횡단보도 후보 + 거리(라디안)까지 같이 반환
ind_array, dist_array = tree.query_radius(acc_rad, r=radius_rad, return_distance=True)

rows = []
for ai, (cw_inds, cw_dists_rad) in enumerate(zip(ind_array, dist_array)):
    if len(cw_inds) == 0:
        continue

    a_id = acc_v.loc[ai, acc_id]
    # meters로 변환
    cw_dists_m = cw_dists_rad * EARTH_RADIUS_M

    for ci, dm in zip(cw_inds, cw_dists_m):
        rows.append({
            "accident_id": a_id,
            "crosswalk_id": cw_v.loc[ci, cw_id],
            "distance_m": float(dm),
        })

out = pd.DataFrame(rows)

# 같은 (accident_id, crosswalk_id) 중복이 생길 가능성은 낮지만 안전하게 제거
out = out.drop_duplicates(subset=["accident_id", "crosswalk_id"]).reset_index(drop=True)

# 보기 좋게 정렬 (가까운 것부터)
out = out.sort_values(["accident_id", "distance_m"]).reset_index(drop=True)

out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print("saved:", OUT_CSV, "rows:", len(out))
out.head()


saved: acc_cw_within_100m.csv rows: 1435


,accident_id,crosswalk_id,distance_m
0,6778766,CW_040719,24.475310
1,6778766,CW_040721,51.794083
2,6778766,CW_040718,53.561044
3,6778766,CW_041543,53.561044
4,6778766,CW_041544,53.561044
